In [ ]:
from pynq import Overlay
from imutils.video import VideoStream
from collections import deque as d
import imutils
import cv2
import copy
import math

def compare_bounding_boxes(b1, b2):
    """
    Returns true if b1 is bigger than b2, otherwise false
    """
    (x1, y1, w1, h1) = b1
    (x2, y2, w2, h2) = b2

    c1 = w1 + w1 + h1 + h1
    c2 = w2 + w2 + h2 + h2

    return c1 > c2


def sendProbabilityInBinaryToMCU(probability):
    numbits = 3
    possibleRepresentationOfAllTheBits = math.pow(2, numbits)
    step = 100/possibleRepresentationOfAllTheBits

    for i in range(0, 8):
        if inRange(probability, i * step, (i + 1) * step):
            bitString = '{0:03b}'.format(i)
            print(bitString)

            writeToGPIO(0, 4, bitString[0])
            writeToGPIO(1, 7, bitString[1])
            writeToGPIO(2, 33, bitString[2])
            
            writeToLED(0, bitString[0])
            writeToLED(1, bitString[1])
            writeToLED(2, bitString[2])

            break

def writeToGPIO(led_number, gpio_number, value):
    if (value is "0"):
        if gpio_number is 4:
            gpio4.write(0)
        elif gpio_number is 7:
            gpio7.write(0)
        elif gpio_number is 33:
            gpio33.write(0)
    else:
        if gpio_number is 4:
            gpio4.write(1)
        elif gpio_number is 7:
            gpio7.write(1)
        elif gpio_number is 33:
            gpio33.write(1)
            
def writeToLED(led_number, value):
    # todo: implement writing to leds with the new base overlay
    pass


def inRange(number, a, b):
    if number >= a and number < b:
        return True
    else:
        return False

# Get the overlay
ol = Overlay("/home/xilinx/rosetta/design_1.bit")
ol.download()

# Get the GPIO pins
gpio4 = ol.axi_gpio_0.channel1[4]
gpio7 = ol.axi_gpio_0.channel1[7]
gpio33 = ol.axi_gpio_0.channel2[7]

# Initialize the pins to have 0 in value
gpio4.write(0)
gpio7.write(0)
gpio33.write(0)

min_area = 500
videopath = "eirik2.mov"
oldBoundingBox = (0, 0, 0, 0)


vs = cv2.VideoCapture(videopath)
vs.set(3,320)
vs.set(4,240)


oldsize = -1
oldpos = (-1,-1,-1,-1)
x,y,w,h = 0,0,0,0

# initialize the first frame in the video stream
firstFrame = None

# Dictionary containing a circular framebuffer and a circular list containing 1 if the frame at the
# same index showed something incoming or 0 if the frame at the same index didn't show something
# incoming
BUFFERSIZESMALL = 5
BUFFERSIZELONG  = 25
framebuffer = {
        # frameBuffer contains area of rectangles
        'frameBuffer'  : d(maxlen = BUFFERSIZELONG, iterable = [0 for x in range(BUFFERSIZELONG) ]),
        'incomingshort': d(maxlen = BUFFERSIZESMALL,iterable = [0 for x in range(BUFFERSIZESMALL)]),
        'incominglong' : d(maxlen = BUFFERSIZELONG, iterable = [0 for x in range(BUFFERSIZELONG) ])
        }

# loop over the frames of the video
while True:
    frame = vs.read()[1]
    if frame is not None:
        frame = imutils.resize(frame, width=320)
    
    debugframe = copy.deepcopy(frame)
    
    
    # Continue reading the file forever
    if frame is None:
        vs = cv2.VideoCapture(videopath)
        continue
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9,9), 0)

    if firstFrame is None:
        firstFrame = gray
        continue

    #compute the absolute difference between the current frame and the first frame
    frameDelta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(frameDelta, 10, 255, cv2.THRESH_BINARY)[1]

    # dilate the thresholded image to fill in holes, then find contours on thresholded image
    dilation_kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    thresh = cv2.erode(thresh, dilation_kernel, iterations=1)
    thresh = cv2.dilate(thresh, dilation_kernel,iterations=3)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if imutils.is_cv2() else cnts[1]

    # store error values to find the best match later
    epsilon = []
    # Use this to keep track of the previous bounding box
    newBoundingBox = (0, 0, 0, 0)
    rectangles = []

    # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < min_area:
            continue

        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(debugframe, (x, y), (x + w, y + h), (0, 0, 255), 2)

        newBoundingBox = (x, y, w, h)
        
        rectangles.append((x,y, w, h))



    best = (0,0,0,0)

    if len(rectangles) >= 2:
        bestError = 9999
        bestRectangle = None
        for coordinateNew in rectangles:
            # Normalizing, so we compare the center point of the box
            # this ignores size differences (which can cause the
            # interpretation of position to be skewed)
            deltaX = abs((oldpos[0]+oldpos[2]/2) - (coordinateNew[0] + coordinateNew[2]/2))
            deltaY = abs((oldpos[1]+oldpos[3]/2) - (coordinateNew[1] + coordinateNew[3]/2))
            error = deltaX + deltaY

            if error < bestError:
                bestError = error
                x, y, w, h = coordinateNew[0], coordinateNew[1], coordinateNew[2], coordinateNew[3]
                newBoundingBox = (coordinateNew[0], coordinateNew[1], coordinateNew[2], coordinateNew[3])
                best = (x,y,w,h)

        (x,y,w,h) = best
        oldpos = best
        cv2.rectangle(debugframe, (x, y), (x + w, y + h), (0, 255, 0), 2)
    elif len(rectangles) == 1:
        newBoundingBox = (rectangles[0][0], rectangles[0][1], rectangles[0][2], rectangles[0][3])
        x, y, w, h = rectangles[0][0], rectangles[0][1], rectangles[0][2], rectangles[0][3]
        cv2.rectangle(debugframe, (x, y), (x + w, y + h), (0, 255, 0), 2)
        best = (x,y,w,h)
        oldpos = newBoundingBox
    else:
        oldpos = oldpos


    if compare_bounding_boxes(newBoundingBox, oldBoundingBox):
        framebuffer['incomingshort'].append(1)
        if(sum(framebuffer['incomingshort']) >= 3):
            framebuffer['incominglong'].append(1)
        framebuffer['frameBuffer'].append(2*newBoundingBox[2] + 2*newBoundingBox[3])
    else:
        framebuffer['incomingshort'].append(0)
        framebuffer['incominglong' ].append(0)
        framebuffer['frameBuffer'  ].append(2*newBoundingBox[2] + 2*newBoundingBox[3])

    totArea         = 0
    prob            = 0
    positivesamples = 0
    for i in range(BUFFERSIZELONG):
        totArea += framebuffer['frameBuffer'][i]
        if(framebuffer['incominglong'][i] == 1):
            prob += framebuffer['frameBuffer'][i]
            positivesamples += 1


    negativesamples = BUFFERSIZELONG - positivesamples


    if(totArea != 0 and positivesamples > 3 and negativesamples < 20):
        prob /= totArea
    else:
        prob= 0

    sendProbabilityInBinaryToMCU(prob * 100)


    oldBoundingBox = newBoundingBox
    
    firstFrame = gray


